### The TRIQS library beyond impurity solvers: from simple mean-field theory to TRILEX


*Thomas Schäfer, Erwin-Schrödinger Fellow at Collège de France and École Polytechnique*<br>
TRIQS meeting 2019, 12th - 14th June 2019, Collège de France

**Abstract**: Without a doubt the impurity solvers available as TRIQS applications are serving as power horses in many working groups. In my talk, I will convince you that the TRIQS library is a valuable and easy to use tool also in the development and implementation of many-body algorithms. Starting from the calculation of a non-interacting susceptibility, I will demonstrate how the random phase approximation can be implemented elegantly and applied to the 2D Hubbard model on a square lattice. Afterwards, the implementation of a cutting-edge many-body method, TRILEX, will be discussed.

In [ ]:
from pytriqs.plot.mpl_interface import plt
import numpy as np
from numpy import pi
from pytriqs.lattice import BravaisLattice, BrillouinZone
from pytriqs.gf import MeshBrillouinZone, MeshImFreq, MeshImTime, Gf, MeshProduct, \
                       Idx, inverse, make_adjoint_mesh, make_gf_from_fourier,      \
                       make_gf_from_inverse_fourier
from pytriqs.lattice.tight_binding import TightBinding, dos, energies_on_bz_path
from pytriqs.archive import HDFArchive
from pytriqs.plot.mpl_interface import oplot, oploti, oplotr
from scipy.optimize import fsolve, brentq
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (4,4)

Parameter definition
---

In [ ]:
beta = 4.   # inverse temperature
mu   = 0.   # chemical potential

n_k  = 64   # number of (linear) fermionic momenta
n_iw = 20   # number of (positive) fermionic Matsubara frequencies
n_q  = n_k  # number of (linear) bosonic momenta
n_iW = n_iw # number of (positive) bosonic Matsubara frequencies

In [ ]:
# define the Bravais lattice: a square lattice in 2D
BL = BravaisLattice(units = [(1, 0), (0, 1)])

# define the tight-binding model, i.e., the hopping parameters
t = -1.0               # nearest neighbor hopping in real space

hop= {  (1,  0)  :  [[ t]],       
        (-1, 0)  :  [[ t]],     
        (0,  1)  :  [[ t]], 
        (0, -1)  :  [[ t]]}

TB = TightBinding(BL, hop)

In [ ]:
# Compute the density of states
d = dos(TB, n_kpts=800, n_eps=101, name='')[0]

In [ ]:
oplot(d,'-o')
plt.xlim(-4.5, 4.5)
plt.ylim(0., 0.4)
plt.xlabel(r"$\varepsilon$", fontsize=18)
plt.ylabel(r"$\rho_{0}(\varepsilon)$", fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.show()

In [ ]:
# calculation of the dispersion for a given k-point
# to be put into the TRIQS-library
def eps(BL, TB, k):    
    return energies_on_bz_path(TB, \
            [k[i]*1./(2.*pi) for i in range(0,BL.dim)], \
            [k[i]*1./(2.*pi) for i in range(0,BL.dim)], \
            1)[0,0]

In [ ]:
k_mesh    = MeshBrillouinZone(BrillouinZone(BL), n_k)
iw_mesh   = MeshImFreq(beta, 'Fermion', n_iw)
k_iw_mesh = MeshProduct(k_mesh, iw_mesh)

q_mesh    = MeshBrillouinZone(BrillouinZone(BL), n_q)
iW_mesh   = MeshImFreq(beta, 'Boson', n_iW)
q_iW_mesh = MeshProduct(q_mesh, iW_mesh)

non-interacting lattice Green function
--

#### $G_{0}=\frac{1}{i\omega_{n} - \varepsilon_{\mathbf{k}} + \mu}$

In [ ]:
G0 = Gf(mesh=k_iw_mesh, target_shape=[])

for k, iw in k_iw_mesh:
    G0[k,iw] = 1. / (iw - eps(BL,TB,k) + mu)

In [ ]:
print G0

In [ ]:
k_linear = np.linspace(-pi, pi, 10. * n_k, endpoint=True)
kx, ky = np.meshgrid(k_linear, k_linear)

to_plot = lambda kx, ky: -1. / pi * G0((kx,ky,0),0).imag

In [ ]:
gs=plt.GridSpec(1,1)
plt.set_cmap('terrain')

ax = plt.subplot(gs[0],aspect="equal")
plt.pcolormesh(kx, ky, np.vectorize(to_plot)(kx, ky))
plt.colorbar(shrink=0.8)
plt.xlabel(r'$k_{x}$', fontsize=18)
plt.ylabel(r'$k_{y}$', fontsize=18)
plt.xlim(-pi,pi)
plt.ylim(-pi,pi)
plt.xticks([-np.pi, 0, np.pi],[r"$-\pi$", r"0", r"$\pi$"], fontsize=18)    
plt.yticks([-np.pi, 0, np.pi],[r"$-\pi$", r"0", r"$\pi$"], fontsize=18)
plt.title(r'-$1/\pi$ Im $G(\mathbf{k}, i\omega_{0})$', fontsize=20)
plt.show()

## non-interacting susceptibility (Lindhard bubble)

#### $ \chi_0(\mathbf{q}, i\Omega_n) = 
    -\frac{1}{\beta}\sum\limits_{\mathbf{k}, m} 
    G_0(\mathbf{k}, i\omega_m)G_0(\mathbf{k}+\mathbf{q}, i\omega_m + i\Omega_n) $

#### $  \chi_0(\mathbf{r},\tau) = G_0(\mathbf{r},\tau)G_0(-\mathbf{r},\beta -\tau) $ 

#### $\chi_0(\mathbf{q}, i\Omega_{n}) \equiv 
  \mathcal{F}_{\{\mathbf{r},\tau\} \rightarrow \{\mathbf{q}, i\Omega_n\}} 
  \big\{ \chi_0(\mathbf{r}, \tau) \big\}$

In [ ]:
r_mesh     = make_adjoint_mesh(k_mesh)
tau_mesh   = make_adjoint_mesh(iw_mesh)
r_tau_mesh = MeshProduct(r_mesh, tau_mesh)
r_iw_mesh  = MeshProduct(r_mesh, iw_mesh)

G0_r_iw  = Gf(mesh=r_iw_mesh, target_shape=[])
G0_r_tau = Gf(mesh=r_tau_mesh, target_shape=[])

for iw in iw_mesh:
    G0_r_iw[:,iw] = make_gf_from_fourier(G0[:,iw])

for r in r_mesh:
    G0_r_tau[r,:] = make_gf_from_fourier(G0_r_iw[r,:])

In [ ]:
G0_minus_r_minus_tau = G0_r_tau.copy()
G0_minus_r_minus_tau.zero()
for r, tau in r_tau_mesh:
    minus_r = -(np.array(r.value, dtype=np.int))
    minus_tau = beta - tau
    G0_minus_r_minus_tau[r,tau] = G0_r_tau(minus_r, minus_tau)

In [ ]:
chi0_r_tau = G0_r_tau * G0_minus_r_minus_tau

In [ ]:
print chi0_r_tau

In [ ]:
chi0_r_iW = Gf(mesh=MeshProduct(r_mesh, iW_mesh), target_shape=[])
chi0 = Gf(mesh=q_iW_mesh, target_shape=[])

for r in r_mesh:
    chi0_r_iW[r,:] = make_gf_from_fourier(chi0_r_tau[r,:])

for iW in iW_mesh:
    chi0[:,iW] = make_gf_from_fourier(chi0_r_iW[:,iW])

In [ ]:
q_linear = np.linspace(0, 2. * pi, 10. * n_q, endpoint=True)
qx, qy = np.meshgrid(q_linear, q_linear)

to_plot = lambda qx, qy: chi0((qx,qy,0),0).real

In [ ]:
ax = plt.subplot(gs[0],aspect="equal")
plt.pcolormesh(qx, qy, np.vectorize(to_plot)(qx, qy))
plt.colorbar(shrink=0.8)
plt.xlabel(r'$q_{x}$', fontsize=18)
plt.ylabel(r'$q_{y}$', fontsize=18)
plt.xlim(0,2*pi)
plt.ylim(0,2*pi)
plt.xticks([0, pi/2, pi, 3.*pi/2, 2*pi],[r"0", r"$\pi/2$", r"$\pi$", r"$3\pi/2$", r"$2\pi$"], fontsize=18)    
plt.yticks([0, pi/2, pi, 3.*pi/2, 2*pi],[r"0", r"$\pi/2$", r"$\pi$", r"$3\pi/2$", r"$2\pi$"], fontsize=18)
plt.title(r'Re $\chi_{0}(\mathbf{q}, i\Omega_{0})$', fontsize=20)
plt.show()

In [ ]:
%%timeit -r 1
for r,tau in r_tau_mesh:
    minus_r = -(np.array(r.value, dtype=np.int))
    minus_tau = beta - tau
    G0_minus_r_minus_tau[r,tau] = G0_r_tau(minus_r, minus_tau)

for iw in iw_mesh:
    G0_r_iw[:,iw] = make_gf_from_fourier(G0[:,iw])

for r in r_mesh:
    G0_r_tau[r,:] = make_gf_from_fourier(G0_r_iw[r,:])

chi0_r_tau = G0_r_tau * G0_minus_r_minus_tau

for r in r_mesh:
    chi0_r_iW[r,:] = make_gf_from_fourier(chi0_r_tau[r,:])
for iW in iW_mesh:
    chi0[:,iW] = make_gf_from_fourier(chi0_r_iW[:,iW])

C++ implementation
---

In [ ]:
%reload_ext cpp2py.magic

In [ ]:
%%cpp2py -C pytriqs
#include <triqs/gfs.hpp>
using namespace triqs::gfs;
        
using g_k_iw_type     = gf_view<cartesian_product<brillouin_zone, imfreq>, scalar_valued>;
using chi_r_tau_type  = gf<cartesian_product<cyclic_lattice, imtime>, scalar_valued>;

triqs::clef::placeholder<0> r_;
triqs::clef::placeholder<1> tau_;

g_k_iw_type bubble(g_k_iw_type g0) {
    auto g0_r_tau = make_gf_from_fourier<0,1>(g0);
    
    auto [r_mesh, tau_mesh] = g0_r_tau.mesh();
    double beta = tau_mesh.domain().beta;
    
    auto tau_mesh_bosonic = gf_mesh<imtime>{beta, Boson, tau_mesh.size()};
    
    auto chi0 = chi_r_tau_type{{r_mesh, tau_mesh_bosonic}};

    chi0[r_, tau_] << g0_r_tau(-r_, beta - tau_) * g0_r_tau(r_, tau_); 

    return make_gf_from_fourier<0,1>(chi0);
}

In [ ]:
%%timeit -r 1
chi0 = bubble(G0)

In [ ]:
ax = plt.subplot(gs[0],aspect="equal")
plt.pcolormesh(qx, qy, np.vectorize(to_plot)(qx, qy))
plt.colorbar(shrink=0.8)
plt.xlabel(r'$q_{x}$', fontsize=18)
plt.ylabel(r'$q_{y}$', fontsize=18)
plt.xlim(0,2*pi)
plt.ylim(0,2*pi)
plt.xticks([0, pi/2, pi, 3.*pi/2, 2*pi],[r"0", r"$\pi/2$", r"$\pi$", r"$3\pi/2$", r"$2\pi$"], fontsize=18)    
plt.yticks([0, pi/2, pi, 3.*pi/2, 2*pi],[r"0", r"$\pi/2$", r"$\pi$", r"$3\pi/2$", r"$2\pi$"], fontsize=18)
plt.title(r'Re $\chi_{0}(\mathbf{q}, i\Omega_{0})$', fontsize=20)
plt.show()

Random phase approximation (RPA)
----

Bethe-Salpeter equation (with our definition of $\chi_{0}$):
#### $ \chi = \chi_0 + \chi_0 \Gamma \chi = \frac{2\chi_0}{1 - \Gamma \chi_0} $

random phase approximation: 
#### $\Gamma = U$

leads to
#### $\chi^{RPA}_{sp}(\mathbf{q},i\Omega_{n})=\frac{2\chi_{0}(\mathbf{q},i\Omega_{n})}{1 - U\chi_{0}(\mathbf{q},i\Omega_{n})}$

In [ ]:
U = 2.
chi_RPA = 2. * chi0 * inverse(1. - U * chi0)

In [ ]:
to_plot = lambda qx, qy: chi_RPA((qx,qy,0),0).real

In [ ]:
ax = plt.subplot(gs[0],aspect="equal")
plt.pcolormesh(qx, qy, np.vectorize(to_plot)(qx, qy))
plt.colorbar(shrink=0.8)
plt.xlabel(r'$q_{x}$', fontsize=18)
plt.ylabel(r'$q_{y}$', fontsize=18)
plt.xlim(0,2*pi)
plt.ylim(0,2*pi)
plt.xticks([0, pi/2, pi, 3.*pi/2, 2*pi],[r"0", r"$\pi/2$", r"$\pi$", r"$3\pi/2$", r"$2\pi$"], fontsize=18)    
plt.yticks([0, pi/2, pi, 3.*pi/2, 2*pi],[r"0", r"$\pi/2$", r"$\pi$", r"$3\pi/2$", r"$2\pi$"], fontsize=18)
plt.title(r'Re $\chi^{RPA}(\mathbf{q}, i\Omega_{0})$', fontsize=20)
plt.show()